In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
files.upload()

In [ ]:
#나눔바른고딕 다운로드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
df = pd.read_csv('vgames2.csv')
df.info()

In [ ]:
#check data size
df.shape

In [ ]:
# check missing data
df.isnull().sum()
#Year, Genre, Publisher missing data

In [ ]:
#중복확인
df.duplicated().sum()

In [ ]:
# unnamed은 필요없는 데이터이므로 지운다
#df를 clean dataframe으로 남기기 위해서 새로운 변수 지정
dfy_col = df.iloc[:,1:]
dfy_col.head(-10)

In [ ]:
# 연도별 게임 트랜드가 있을까 에 대한 분석 시작
# 연도와 장르가 중요하기 때문에 Year열과 Genre열, Publisher 에서 결측치가 있는 행들은 평균값이나 임의의 값을 넣어준다
dfy_col['Year']=dfy_col['Year'].fillna(dfy_col['Year'].mean())
dfy_col['Publisher']= dfy_col['Publisher'].fillna('X')
dfy_col['Genre']= dfy_col['Genre'].fillna('X')
dfy_col.head(-10)

In [ ]:
#Year, 'NA_Sales' ,EU_Sales,JP_Sales,Other_Sales 내 'K', 'M' 변경
dfy_col['NA_Sales'].replace({"M":""}, inplace = True, regex=True)
dfy_col['EU_Sales'].replace({ "M":""},inplace = True, regex=True)
dfy_col['JP_Sales'].replace({"M":""}, regex=True,inplace = True)
dfy_col['Other_Sales'].replace({"M":""}, regex=True,inplace = True)

In [ ]:
ind1 =  dfy_col[dfy_col['NA_Sales'].str.count('K')>0]['NA_Sales'].index
ind2 =  dfy_col[dfy_col['EU_Sales'].str.count('K')>0]['EU_Sales'].index
ind3 =  dfy_col[dfy_col['JP_Sales'].str.count('K')>0]['JP_Sales'].index
ind4 =  dfy_col[dfy_col['Other_Sales'].str.count('K')>0]['Other_Sales'].index

In [ ]:
#NA
ind1_t = []
for t in ind1 :
  ind1_t.append(float(t)*0.001)
a=0
while a < len(ind1_t):
  for i in ind1:
    dfy_col.iloc[i,5] = ind1_t[a]
    a += 1

In [ ]:
#EU
ind2_t = []
for t in ind2 :
  ind2_t.append(float(t)*0.001)
a=0
while a < len(ind2_t):
  for i in ind2:
    dfy_col.iloc[i,6] = ind2_t[a]
    a += 1

In [ ]:
#JP
ind3_t = []
for t in ind3 :
  ind3_t.append(float(t)*0.001)
a=0
while a < len(ind3_t):
  for i in ind3:
    dfy_col.iloc[i,7] = ind3_t[a]
    a += 1

In [ ]:
#Other
ind4_t = []
for t in ind4 :
  ind4_t.append(float(t)*0.001)
a=0
while a < len(ind4_t):
  for i in ind4:
    dfy_col.iloc[i,8] = ind4_t[a]
    a += 1

In [ ]:
#Year, 'NA_Sales' ,EU_Sales,JP_Sales,Other_Sales 타입변경
dfy_col = dfy_col.astype({'Year':'int' })
dfy_col['NA_Sales'] = dfy_col['NA_Sales'].astype(float, errors = 'raise')
dfy_col['JP_Sales'] = dfy_col['JP_Sales'].astype(float, errors = 'raise')
dfy_col['Other_Sales'] = dfy_col['Other_Sales'].astype(float, errors = 'raise')
dfy_col['EU_Sales'] = dfy_col['EU_Sales'].astype(float, errors = 'raise')

In [ ]:
dfy_col.info()

In [ ]:
#M(백만)표시가 있었으므로 단위가 안 붙어 있던 숫자들은 기본단위 M으로 가정하여 백만을 곱해준다
#계속 진행해보니 숫자가 너무 커지는 거 같아서 Sales 들어간 컬럼의 기본단위는 K로 수정
dfy_col['NA_Sales'] = dfy_col['NA_Sales']*1000
dfy_col['EU_Sales'] = dfy_col['EU_Sales']*1000
dfy_col['JP_Sales'] = dfy_col['JP_Sales']*1000
dfy_col['Other_Sales'] = dfy_col['Other_Sales']*1000

In [ ]:
#Sales 합계구하자
dfy_col['Sales'] = dfy_col.eval('NA_Sales+EU_Sales+JP_Sales+Other_Sales')
dfy_col.head(-10)

In [ ]:
#연도별 트렌드를 알기 위해서 판매량 수집
dfy_col.groupby('Genre')['Sales'].mean()

In [ ]:
#그래프
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font='NanumBarunGothic', rc={"axes.unicode_minus":False},style='whitegrid')

In [ ]:
dfy_col.groupby('Year').sum()

In [ ]:
#1980년 이전과 2017년 이후는 데이터가 의미 없으므로 지운다
dfy_col_clean = dfy_col[(dfy_col.Year >= 1980) & (dfy_col.Year<=2016)]
dfy_col_clean = dfy_col_clean.reset_index(drop=True)
dfy_col_clean

In [ ]:
#Genre의 unique 확인
dfy_col_clean['Genre'].unique()

# **1. 지역에따라 선호하는 게임장르**

In [ ]:
dfy_genre = pd.concat([dfy_col_clean['Genre'],dfy_col_clean['NA_Sales'],dfy_col_clean['EU_Sales'],dfy_col_clean['JP_Sales'],dfy_col_clean['Other_Sales'],dfy_col_clean['Sales']],axis = 1)
dfy_genre = dfy_genre.groupby(['Genre']).sum()
dfy_genre

In [ ]:
#지역에 따라 선호하는 장르의 평균이 유의미한 차이가 있는지 검증
import scipy.stats

chi2, pvalue, dof, expected = scipy.stats.chi2_contingency(dfy_genre)
print(f'p-value 값이 {pvalue}이므로 지역과 장르는 충분한 상관관계를 갖는다') 

In [ ]:
# 연도에 따른 각 지역별 출고량을 위한 전처리

df_ys = dfy_col_clean.groupby(by = ['Year','Genre'])['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Sales'].sum().stack()
df_ys = df_ys.reset_index()
df_ys.rename(columns = {'level_2' : 'Region'}, inplace = True)
df_ys.rename(columns = {0 : 'Total_Sales'}, inplace = True)
df_ys.head()

In [ ]:
#지역과 장르에 대한 시각화
plt.figure(figsize = (13,10))
bar_width = 0.15
index = np.arange(13)
b1 = plt.bar(index, dfy_col_clean.groupby('Genre')['NA_Sales'].sum()
             ,bar_width,alpha=0.4, color='red', label='NA_Sales')
b2 = plt.bar(index+bar_width, dfy_col_clean.groupby('Genre')['EU_Sales'].sum()
             ,bar_width,alpha=0.4, color='blue', label='EU_Sales' )
b3 = plt.bar(index+2*bar_width, dfy_col_clean.groupby('Genre')['JP_Sales'].sum()
             ,bar_width,alpha=0.4, color='green', label='JP_Sales' )
b4 = plt.bar(index+3*bar_width, dfy_col_clean.groupby('Genre')['Other_Sales'].sum()
             ,bar_width,alpha=0.4, color='black', label='Other_Sales' )
b4 = plt.bar(index+4*bar_width, dfy_col_clean.groupby('Genre')['Sales'].sum()
             ,bar_width,alpha=0.4, color='pink', label='Sales' )
# x축 위치를 정 가운데로 조정하고 x축의 텍스트를 genre 정보와 매칭
plt.xticks(np.arange(bar_width, 13 + bar_width, 1), dfy_col_clean['Genre'].value_counts().sort_index().index)
# x축, y축 이름 및 범례 설정
plt.xlabel('genre', size = 13)
plt.ylabel('sale', size = 13)
plt.title('지역에 따라 선호하는 게임장르')
plt.legend()
plt.show()

In [ ]:
# 연도에 따른 지역별 판매량 그래프 그리기
plt.figure(figsize=(15, 5))
sns.lineplot(data=df_ys, x="Genre", y="Total_Sales", hue="Region", linewidth=3)
plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
plt.title('장르에 따른 각 지역별 판매량', fontsize=20)

각 지역별로 장르 판매량을 봤을 때 
미국,유럽, 기타 국가들의 경우 Action, Sports, Shooter 순으로 많이 팔렸고
일본의 경우 Role-Playing, Action, Sports 순으로 팔렸다
이 결과는 일본을 제외한 나머지 국가들은 Action 게임을 가장 많이 즐기고
일본은 Role-Playing 게임을 즐긴다는 것을 보여준다.

# **2. 연도별 게임 트랜드 확인**

In [ ]:
df_yg = dfy_col_clean.groupby(['Year', 'Genre'])['Sales'].sum().unstack()
df_yg = df_yg.fillna(0)
df_yg = df_yg.reset_index()
#df_yg = df_yg.stack()
df_yg.info()

In [ ]:
#연도와 장르에 대한 시각화
plt.figure(figsize = (12,10))
plt.title('연도별 게임의 트랜드', fontsize=20)
plt.ylabel('판매량 (K)')
sns.scatterplot(data = dfy_col_clean, x = dfy_col_clean['Year'],y = dfy_col_clean['Sales'], hue = dfy_col_clean['Genre'], 
                style = dfy_col_clean['Genre'], s = (50,900) , size = dfy_col_clean['Sales'])

In [ ]:
plt.figure(figsize = (15,8))
plt.title('연도별 게임의 트랜드', fontsize=20)
plt.ylabel('판매량 (K)')
sns.set_palette("Paired")
genre = df_yg.columns
for i in range(1, len(genre)):
    sns.lineplot(data=df_yg, x='Year', y=genre[i], label=genre[i])

In [ ]:
df_ygs = dfy_col_clean.groupby(['Year', 'Genre'])['Sales'].sum()
df_ygs = df_ygs.reset_index()
df_ygs.head()

In [ ]:
# 판매량이 가장 높은 3개의 장르 리스트 만들기.

df_genre = df_ygs.groupby('Genre')['Sales'].sum().to_frame()
df_genre = df_genre.sort_values(by='Sales', ascending=False).head(3)
df_genre = df_genre.reset_index()
genre_list = df_genre['Genre'].array

# 판매량이 가장 높은 3개의 게임장르의 정보 담기.
df_genre_top3 = pd.DataFrame()

for i in range(len(genre_list)):
    df_g = df_ygs[df_ygs['Genre']==genre_list[i]]
    df_genre_top3 = df_genre_top3.append(df_g)
    
df_genre_top3

In [ ]:
# 연도에 따른 탑3 장르별 판매량 그래프 그리기
plt.figure(figsize=(15, 8))

sns.lineplot(data=df_genre_top3, x="Year", y="Sales", hue="Genre", marker='o', linewidth=3)
plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
plt.title('연도에 따른 TOP3 장르별 판매량', fontsize=20)
plt.ylabel('판매량 (K)')

In [ ]:
#5년 단위 장르별 판매량
df_p = dfy_col_clean.groupby(['Year', 'Genre'])['Genre'].count().unstack()
df_p = df_p.fillna(0)
df_p.head()

In [ ]:
df_p.loc['1980-1984', :] = df_p.loc[[1980, 1981, 1982, 1983, 1984]].sum()
df_p.loc['1985-1989', :] = df_p.loc[[1985, 1986, 1987, 1988, 1989]].sum()
df_p.loc['1990-1994', :] = df_p.loc[[1990, 1991, 1992, 1993, 1994]].sum()
df_p.loc['1995-1999', :] = df_p.loc[[1995, 1996, 1997, 1998, 1999]].sum()
df_p.loc['2000-2004', :]= df_p.loc[[2000, 2001, 2002, 2003, 2004]].sum()
df_p.loc['2005-2009', :]= df_p.loc[[ 2005, 2006, 2007, 2008, 2009]].sum()
df_p.loc['2010-2016', :] = df_p.loc[[2010, 2011, 2012, 2013, 2014, 2015, 2016]].sum()

In [ ]:
df_p = df_p.loc[['1980-1984','1985-1989', '1990-1994','1995-1999', '2000-2004','2005-2009', '2010-2016']]
df_p.reset_index()
df_p

In [ ]:
plt.figure(figsize=(15, 7))
plt.title('시대별(5년) 게임 장르 트렌드', fontsize=15)
plt.ylabel('판매량 (K)')
sns.lineplot(data=df_p, marker='o', linewidth=3)
#sns.lineplot(data=df_genre_top3, x="Year", y="Sales", hue="Genre", marker='o', linewidth=3)

In [ ]:
df_ygss = df_ygs.groupby(['Year'])['Sales'].transform(max) == df_ygs['Sales']
df_ygss_max = df_ygs[df_ygss].reset_index(drop=True)

genre = df_ygss_max['Genre']

plt.figure(figsize=(30, 18))
g = sns.barplot(x='Year', y='Sales', data=df_ygss_max)
index = 0
for value in df_ygss_max['Sales']:
    g.text(index, value + 1, str(genre[index]),size=20, rotation= 90, ha="center")
    index += 1

plt.title("연도별 게임 트랜드", fontsize=30)
plt.xlabel("연도", fontsize=20)
plt.ylabel("총 판매량", fontsize=20)

plt.xticks(rotation=90)
plt.show()

라인 그래프를 통해 각 시대별로 어떤 장르 게임의 판매량이 많았는지 알 수 있다.

skyblue색 실선을 의미하는 Action 장르는 1995-1999 구간 이후로 급격하게 성장하였다.
1990부터 2009년까지 전 장르의 판매량이 증가하였다.
하지만, 2009년 이후로 전반적으로 판매량이 감소하는 추세이다.

바 그래프를 통해서 각 연도 별로 어떤 장르의 게임이 가장 많이 팔렸는지 알 수 있다
2000년도 까지는 여러 장르들이 팔렸지만 2000년 이후 16년 중  2006,2007, 2014, 2016이렇게 4년을 제외하고는 Action게임이 매해 가장 많이 팔렸다

# **3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스**

In [ ]:
df_k = dfy_col_clean.sort_values(by='Sales', ascending=False).head(20)
df_kd =df_k[['Name', 'Year', 'Genre', 'Platform', 'Sales']].sort_values(by = 'Sales', ascending = False).reset_index(drop = True)
df_kd

In [ ]:
plt.figure(figsize=(12, 5))
plt.title('TOP 20 게임의 장르별 판매량', fontsize=20)
sns.barplot(data=df_kd, x="Genre", y="Sales", palette="rainbow", alpha=0.8)

In [ ]:
df_kdp = df_kd.groupby(["Platform"]).sum()
df_kdp = df_kdp.sort_values(by = 'Sales', ascending=False)
df_kdp.head()

In [ ]:
wedgeprops = {'width':0.7, 'edgecolor' : 'w', 'linewidth' : 5}
plt.figure(figsize=(12,10))
plt.pie(df_kdp['Sales'], labels = df_kdp.index.values[:10], autopct='%1.2f%%', startangle=50, wedgeprops = wedgeprops)
plt.axis('equal')
plt.title('TOP20 게임 플랫폼의 판매량', fontsize = 20)
plt.show()

In [ ]:
#5년 단위 플랫폼별 판매량
df_plat = dfy_col_clean.groupby(['Year', 'Platform'])['Platform'].count().unstack()
df_plat = df_plat.fillna(0)
df_plat.head()

In [ ]:
df_plat.loc['1980-1984', :] = df_plat.loc[[1980, 1981, 1982, 1983, 1984]].sum()
df_plat.loc['1985-1989', :] = df_plat.loc[[1985, 1986, 1987, 1988, 1989]].sum()
df_plat.loc['1990-1994', :] = df_plat.loc[[1990, 1991, 1992, 1993, 1994]].sum()
df_plat.loc['1995-1999', :] = df_plat.loc[[1995, 1996, 1997, 1998, 1999]].sum()
df_plat.loc['2000-2004', :] = df_plat.loc[[2000, 2001, 2002, 2003, 2004]].sum()
df_plat.loc['2005-2009', :] = df_plat.loc[[2005, 2006, 2007, 2008, 2009]].sum()
df_plat.loc['2010-2016', :] = df_plat.loc[[2010, 2011, 2012, 2013, 2014, 2015, 2016]].sum()

In [ ]:
#플랫폼을 5년 단위로 나눈다
df_plat = df_plat.loc[['1980-1984','1985-1989','1990-1994','1995-1999', '2000-2004','2005-2009', '2010-2016']]
df_plat.reset_index()
df_plat

In [ ]:
dfy_col_clean_p = dfy_col_clean.groupby(['Year','Platform'])['Sales'].sum()
dfy_col_clean_p = dfy_col_clean_p.reset_index()
dfy_col_clean_p.sort_values(by ='Sales', ascending=False)

In [ ]:
df_plat.plot(kind='bar', figsize=(20, 10), cmap='rainbow', alpha=0.5 )
plt.title('1995-2016 플랫폼별 판매량', fontsize=20)
plt.legend(loc='best')

In [ ]:
df_ygsp = dfy_col_clean_p.groupby(['Year'])['Sales'].transform(max) == dfy_col_clean_p['Sales']
df_ygsp_max = dfy_col_clean_p[df_ygsp].reset_index(drop=True)

platform = df_ygsp_max['Platform']

plt.figure(figsize=(30, 18))
g = sns.barplot(x='Year', y='Sales', data=df_ygsp_max)
index = 0
for value in df_ygsp_max['Sales']:
    g.text(index, value + 1, str(platform[index]),size=20, rotation= 90, ha="center")
    index += 1

plt.title('연도별 플랫폼 트렌드', fontsize=30)
plt.xlabel("연도", fontsize=20)
plt.ylabel('판매량 (K)', fontsize=20)

plt.xticks(rotation=90)
plt.show()

In [ ]:
dfy_p = dfy_col_clean.groupby(['Platform']).sum()
dfy_p = dfy_p.sort_values(by = 'Sales', ascending=False)
dfy_p.head(10)

In [ ]:
wedgeprops = {'width':0.7, 'edgecolor' : 'w', 'linewidth' : 5}
plt.figure(figsize=(12,10))
plt.pie(dfy_p['Sales'][:10], labels = dfy_p.index.values[:10], autopct='%1.2f%%', startangle=50, wedgeprops = wedgeprops)
plt.axis('equal')
plt.title('TOP10 플랫폼의 판매량', fontsize = 20)
plt.show()

In [ ]:
#개별 게임 판매량 Top 20의 장르
dfy_g = df_kd.groupby(['Genre']).sum()
dfy_g = dfy_g.sort_values(by = 'Sales', ascending=False)
dfy_g.head(20)

In [ ]:
wedgeprops = {'width':0.7, 'edgecolor' : 'w', 'linewidth' : 5}
plt.figure(figsize=(12,10))
plt.pie(dfy_g['Sales'], labels = dfy_g.index.values, autopct='%1.2f%%', startangle=50, wedgeprops = wedgeprops)
plt.axis('equal')
plt.title('TOP20 게임의 장르별 판매량', fontsize = 20)
plt.show()

판매량 Top3 중 액션장르와 슈터장르의 붐은 단일 게임의 활약이 아닌 여러 게임의 활약임을 알 수 있었고 스포츠장르는 wil series라는 확고한 강자가 있음을 알 수 있었다 또한 PS시리즈의 플랫폼을 이용한 게임이 많이 팔렸다.

# **4. 다음분기에 어떤 게임을 출시해야 할까?**

1.   전체 게임의 장르별 판매량을 보면 Top3 장르는 Action, Sports, Shooter이다 일본을 제외한 나머지 국가에서 판매량 1,2 ,3위인 장르이다 의외로 일본에서 판매량 1위인 장르인  Role-playing은 Top3안에 들지 못했다

2.   Top20 게임 판매량을 봤을 때는 전체게임 판매량 1위인 Action장르보다 2위인 Sports장르의 게임이 더 많이 팔렸음을 알 수 있다
이 경우는 will sports를 비롯한 will 시리즈의 약진이 돋보인다
3. 플랫폼의 경우 PS시리즈의 판매량이 두들어진다


---


결론 : Top20 게임 판매량을 고려해봤을 때 To3 장르 중 sports 는 will 시리즈가 많을 매출을 차지하고 있습니다 또한 일본에서 가장 많이 팔린 장르인 Role-Playing은 pocketmon 시리즈가 매출의 대분분을 차지하고 있습니다 그렇기 때문에 우리는 북미나 유럽을 타겟으로 Shooter 장르 계열의 Action 게임을 PS시리즈 플랫폼으로 발매하는게 유리합니다.